In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
np.random.seed(1234)

from simulate_module import *

In [2]:
sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}

In [3]:
args = {
    "n_tasks": 15,
    "conservative": True,
    "target_test_size": 0.8,
    "model_type": "lm",
    "base_output_dir": "derived_data"
}

In [5]:
if args["model_type"] == "lm":
    model_class = "lm"
    loss_fn = mse
elif args["model_type"] == "nn":
    model_class = "nn"
    loss_fn =  torch.nn.MSELoss()

In [24]:
for s in sigma_setting:
    # set directory
    if args["conservative"]:
        data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/conservative_derived_data")
    else:
        data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/derived_data")
    data_path = Path(data_path)
    working_path = data_path / s
    working_path.mkdir(parents = True, exist_ok = True)
    # generate data ------------------------------------------------
    np.random.seed(1234)
    f, betas, zs = random_functions(args["n_tasks"], 6,
                                    sigma_between = sigma_setting[s][0],
                                    sigma_within = sigma_setting[s][-1])
    result = []
    for i, fi in enumerate(f):
        x = np.random.uniform(0, 1, 100)
        result.append({
            "task": i,
            "x": x,
            "f": fi(x),
            "y": fi(x) + np.random.normal(0, .1, len(x))
        })
    # save data
    data_df = pd.concat([pd.DataFrame(r) for r in result])
    data_df.to_csv(working_path / "tasks.csv", index = False)
    data_df = data_df.reset_index()
    betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
    betas_df = pd.DataFrame(betas_df)
    betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
    betas_df.to_csv(working_path / "betas.csv", index = False)
    data_dict = data_df.to_dict(orient = "list")
    # bandit selection ------------------------------------------------
    for target_task in range(args["n_tasks"]):
        # prepare input
        input_data = prepare_input(data_dict,
                                   target_task = target_task,
                                   target_test_size = args["target_test_size"],
                                  preprocess = True)
        pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                               index = False)
        # bandit selection
        losses, alpha, beta, bandit_selects, pi, bl = bandit_source_train(input_data = input_data,
                                                                          model = model_class,
                                                                          batch_size = 8,
                                                                          decay_rate = .5,
                                                                          n_it = 100,
                                                                          loss_fn =  loss_fn,
                                                                          conservative = args["conservative"])
        # save outputs
        output_dir = working_path / f"target_{target_task}_{args['target_test_size']}"
        save_files(output_dir, alpha, beta, losses, bandit_selects, pi, bl)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)

try one script

In [4]:
def basis(x, knots=None, degree = 10):
    if knots is None:
        knots = np.linspace(0, 1, 10)
        
    H = [np.ones(len(x))]
    for k in range(len(knots)):
        for j in range(1, degree + 1):
            H.append(pos(x - knots[k]) ** j)
    
    return np.vstack(H).T
def random_functions(n_tasks, k_clusters, sigma_between, sigma_within):
    np.random.seed(1234)
    betas, zs = gaussian_mixture(n_tasks, k_clusters, sigma_between, sigma_within)
    functions = []
    for beta in betas:
        functions.append(spline_fun(beta))
        
    return functions, betas, zs

def gaussian_mixture(n_samples, k_clusters, sigma_between, sigma_within, dim = 101):
    means = np.random.normal(0, sigma_between, (k_clusters, dim))
    cluster_props = np.random.dirichlet(k_clusters * [1.5])
    
    betas, zs = [], []
    for task in range(n_samples):
        z = np.random.choice(range(k_clusters), p = cluster_props)
        delta = np.random.normal(0, sigma_within, (1, dim))
        betas.append(means[z] + delta)
        zs.append(z)
        
    return np.vstack(betas), zs
        
def spline_fun(beta):
    def f(x):
        return np.matmul(basis(x), beta)
    return f
    



In [5]:
base_dir = Path("degree10_0711")
base_dir.mkdir(exist_ok = True)
metadata = []
i = 0
for conservative in [True]:
    for target_train_size in [.9, .5]:#np.linspace(0.2,0.8,4):
        for s in sigma_setting:
            for target_task in range(15):
                for decay_rate in [0.5]:
                    for model_type in ["nn"]:
                        for lr in [1e-4, 2e-4]:
                            for n_epochs in [10, 50, 100]:
                                for sample_size in [100, 500]:
                                    for batch_size in [8, 16]:
                                        metadata.append({
                                            "path": "exp" + str(i),
                                            "n_tasks": 15,
                                            "conservative": conservative,
                                            "target_train_size": target_train_size,
                                            "model_type": model_type,
                                            "s": s,
                                            "target_task": target_task,
                                            "decay_rate": decay_rate,
                                            "sample_size": sample_size,
                                            "lr": lr,
                                            "n_epochs": n_epochs,
                                            "batch_size": batch_size
                                        })
                                        i += 1

metadf = pd.concat([pd.DataFrame(m, index=[i]) for i, m in enumerate(metadata)])
metadf.to_csv( base_dir / "metadata.csv")

sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}



In [6]:
metadf.tail()

,path,n_tasks,conservative,target_train_size,model_type,s,target_task,decay_rate,sample_size,lr,n_epochs,batch_size
2155,exp2155,15,True,0.5,nn,low_bw,14,0.5,500,0.0002,50,16
2156,exp2156,15,True,0.5,nn,low_bw,14,0.5,100,0.0002,100,8
2157,exp2157,15,True,0.5,nn,low_bw,14,0.5,100,0.0002,100,16
2158,exp2158,15,True,0.5,nn,low_bw,14,0.5,500,0.0002,100,8
2159,exp2159,15,True,0.5,nn,low_bw,14,0.5,500,0.0002,100,16


In [ ]:
for i, args in enumerate(metadata):
    working_path = Path(base_dir / args["path"])
    working_path.mkdir(parents = True, exist_ok = True)
    
    if args["model_type"] == "lm":
        loss_fn = mse
    elif args["model_type"] == "nn":
        loss_fn =  torch.nn.MSELoss()
    
    
    # generate data ------------------------------------------------
    np.random.seed(1234)
    f, betas, zs = random_functions(args["n_tasks"], 6,
                                    sigma_between = sigma_setting[args["s"]][0],
                                    sigma_within = sigma_setting[args["s"]][-1])
    result = []
    for i, fi in enumerate(f):
        x = np.random.uniform(0, 1, args["n_epochs"])
        result.append({
            "task": i,
            "x": x,
            "f": fi(x),
            "y": fi(x) + np.random.normal(0, .1, len(x))
        })
    # save data
    data_df = pd.concat([pd.DataFrame(r) for r in result])
    data_df.to_csv(working_path.joinpath("tasks.csv"), index = False)
    data_df = data_df.reset_index()
    betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
    betas_df = pd.DataFrame(betas_df)
    betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
    betas_df.to_csv(working_path / "betas.csv", index = False)
    data_dict = data_df.to_dict(orient = "list")
    
    input_data = prepare_input(data_dict,
                               target_task = args["target_task"],
                                target_train_size = args["target_train_size"],
                                preprocess = True)
    pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                               index = False)
        # bandit selection
    losses, alpha, beta, bandit_selects, pi, bl = bandit_source_train(input_data = input_data,
                                                                          model_class = args["model_type"],
                                                                          batch_size = args["batch_size"],
                                                                          decay_rate = args["decay_rate"],
                                                                          n_it = 100,
                                                                          loss_fn =  loss_fn,
                                                                          conservative = args["conservative"],
                                                                     save_path = working_path,
                                                                     lr = args["lr"],
                                                                     n_epochs = args["n_epochs"])
        # save outputs
    save_files(working_path, alpha, beta, losses, bandit_selects, pi, bl)
    print(args)
    print(str(bl["bandit_final"]), str(bl["target_train"]))

{'path': 'exp0', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 10, 'batch_size': 8}
[0.8505192399024963] [1.8676700592041016]
{'path': 'exp1', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 10, 'batch_size': 16}
[0.897818386554718] [1.6272337436676025]
{'path': 'exp2', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 10, 'batch_size': 8}
[0.8367719650268555] [1.8819787502288818]
{'path': 'exp3', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 10, 'batch_size

{'path': 'exp32', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 1, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 100, 'batch_size': 8}
[0.5521894693374634] [0.7154635190963745]
{'path': 'exp33', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 1, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 100, 'batch_size': 16}
[0.868575930595398] [0.6988736391067505]
{'path': 'exp34', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 1, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 100, 'batch_size': 8}
[0.7967246770858765] [0.7628424763679504]
{'path': 'exp35', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 1, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 100, 'ba

{'path': 'exp63', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 10, 'batch_size': 16}
[0.9800987243652344] [0.20598343014717102]
{'path': 'exp64', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 50, 'batch_size': 8}
[0.041537750512361526] [0.18917439877986908]
{'path': 'exp65', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 50, 'batch_size': 16}
[0.43515080213546753] [0.09633823484182358]
{'path': 'exp66', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 50,

{'path': 'exp94', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 100, 'batch_size': 8}
[0.12882345914840698] [0.49857887625694275]
{'path': 'exp95', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 100, 'batch_size': 16}
[0.4831565022468567] [0.5024705529212952]
{'path': 'exp96', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 4, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 10, 'batch_size': 8}
[0.967121958732605] [0.18964684009552002]
{'path': 'exp97', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 4, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 10, 'b

{'path': 'exp125', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 50, 'batch_size': 16}
[0.34148108959198] [0.21890202164649963]
{'path': 'exp126', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 50, 'batch_size': 8}
[0.9186426401138306] [0.11802928149700165]
{'path': 'exp127', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 50, 'batch_size': 16}
[0.28567206859588623] [0.21048693358898163]
{'path': 'exp128', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.9, 'model_type': 'nn', 's': 'high_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 100

...

{'path': 'exp1861', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 10, 'batch_size': 16}
[1.2558648586273193] [0.1963844895362854]
{'path': 'exp1862', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 10, 'batch_size': 8}
[1.2567079067230225] [0.21781852841377258]
{'path': 'exp1863', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 10, 'batch_size': 16}
[1.2616747617721558] [0.18157146871089935]
{'path': 'exp1864', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 50,

{'path': 'exp1892', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 100, 'batch_size': 8}
[0.7985070943832397] [0.5303536057472229]
{'path': 'exp1893', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 100, 'batch_size': 16}
[0.2791841924190521] [0.5518673658370972]
{'path': 'exp1894', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 100, 'batch_size': 8}
[0.2176171839237213] [0.5370939373970032]
{'path': 'exp1895', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 3, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 100

{'path': 'exp1923', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 10, 'batch_size': 16}
[0.8737277984619141] [0.42210638523101807]
{'path': 'exp1924', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 50, 'batch_size': 8}
[0.14837506413459778] [0.2788981795310974]
{'path': 'exp1925', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0001, 'n_epochs': 50, 'batch_size': 16}
[0.4286423325538635] [0.30628636479377747]
{'path': 'exp1926', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 5, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 50

{'path': 'exp1954', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 6, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 100, 'batch_size': 8}
[0.07519789040088654] [0.15668682754039764]
{'path': 'exp1955', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 6, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0001, 'n_epochs': 100, 'batch_size': 16}
[0.07383471727371216] [0.20910343527793884]
{'path': 'exp1956', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 6, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 10, 'batch_size': 8}
[0.7277910709381104] [0.22980524599552155]
{'path': 'exp1957', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 6, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs':

{'path': 'exp1985', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 7, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 50, 'batch_size': 16}
[0.07531044632196426] [0.25948476791381836]
{'path': 'exp1986', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 7, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 50, 'batch_size': 8}
[0.3989143967628479] [0.3030394911766052]
{'path': 'exp1987', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 7, 'decay_rate': 0.5, 'sample_size': 500, 'lr': 0.0002, 'n_epochs': 50, 'batch_size': 16}
[0.09558591991662979] [0.2198684960603714]
{'path': 'exp1988', 'n_tasks': 15, 'conservative': True, 'target_train_size': 0.5, 'model_type': 'nn', 's': 'low_bw', 'target_task': 7, 'decay_rate': 0.5, 'sample_size': 100, 'lr': 0.0002, 'n_epochs': 10